### Preamble

In [1]:
import $ivy.`org.scalatest::scalatest:3.2.16`
import org.scalatest.{Filter => _, _}, flatspec._, matchers._

import $ivy.$                                

import org.scalatest.{Filter => _, _}, flatspec._, matchers._


# Topic 6. Higher-order functions and modular programming

In this notebook, we will see that higher-order functions (HOFs) are essential modularity devices, and we will introduce the most common higher-order functions that operate over many different data structures.

### References

[__Programming in Scala, 
A comprehensive step-by-step guide__](https://www.artima.com/shop/programming_in_scala_3ed) Third Edition.
by Martin Odersky, Lex Spoon, and Bill Venners. 

- Chapter 6. Functional objects
- Chapter 8. Functions and closures
- Chapter 16. Working with Lists
- Chapter 23. For Expressions Revisited

[__Functional programming in Scala__](https://www.manning.com/books/functional-programming-in-scala), by Paul Chiusano and Runar Bjarnason.

- Chapter 3. Functional data structures

[__Functional programming simplified__](https://alvinalexander.com/downloads/fpsimplified-free-preview.pdf), by Alvin Alexander.

- Chapter 22. Functions Are Variables, Too
- Chapter 23. Using Methods As If They Were Functions
- Chapter 24. How to Write Functions That Take Functions as Input Parameters
- Chapter 25. How to Write a ‘map’ Function
- Chapter 27. Functions Can Have Multiple Parameter Groups
- Chapter 28. Partially-Applied Functions (and Currying)

## `FoldRight`: divide and conquer

HOFs shine when the time comes to break monoliths. For instance, let's consider the following two functions:

In [2]:
// sum of elements
def sum(l: List[Int]): Int = 
    l match 
        case Nil => ??? : Int 
        case head :: tail => 
            val tailSol: Int = sum(tail)
            ??? : Int

defined function sum

In [18]:
// sum of elements
def sum(l: List[Int]): Int = 
    dyv(l)(
        ??? : Int,  
        (head: Int, tailSol: Int) => 
            ??? : Int
    )

defined function sum

In [19]:
// sum of elements
def sum(l: List[Int]): Int = 
    dyv(l)(
        0 : Int,  
        (head: Int, tailSol: Int) => 
            head + tailSol : Int
    )

defined function sum

In [21]:
// sum of elements
def sum(l: List[Int]): Int = 
    dyv(l)(0, (head, tailSol) => head + tailSol)

defined function sum

In [22]:
// sum of elements
def sum(l: List[Int]): Int = 
    dyv(l)(0, _+_)

defined function sum

In [5]:
def last[A](l: List[A]): Option[A] = 
    l match 
        case Nil => ??? : Option[A]
        case head :: tail => 
            val tailSol: Option[A] = last(tail)
            ??? : Option[A]

defined function last

In [17]:
def last[A](l: List[A]): Option[A] = 
    dyv(l)(
        ??? : Option[A],
        (head: A, tailSol: Option[A]) => 
            ??? : Option[A]
    )

defined function last

In [17]:
def last[A](l: List[A]): Option[A] = 
    dyv(l)(
        None : Option[A],
        (head: A, tailSol: Option[A]) => 
            if tailSol == None then Some(head)
            else tailSol: Option[A]
    )

defined function last

In [ ]:
def member[A](l: List[A], a: A): Boolean = 
    l match 
        case Nil => ??? : Boolean
        case head :: tail => 
            val tailSol: Boolean = member(tail, a)
            ??? : Boolean

In [43]:
def member[A](l: List[A], a: A): Boolean = 
    l match 
        case Nil => false : Boolean
        case head :: tail => 
            val tailSol: Boolean = member(tail, a)
            head == a || tailSol : Boolean

defined function member

In [47]:
def member[A](l: List[A], a: A): Boolean = 
    l.foldRight(??? : Boolean)(
        (head: A, tailSol: Boolean) => 
            ??? : Boolean)
/*
    l match 
        case Nil => false : Boolean
        case head :: tail => 
            val tailSol: Boolean = member(tail, a)
            head == a || tailSol : Boolean
*/

defined function member

In [48]:
def member[A](l: List[A], a: A): Boolean = 
    l.foldRight(false : Boolean)(
        (head: A, tailSol: Boolean) => 
            head == a || tailSol : Boolean)
/*
    l match 
        case Nil => false : Boolean
        case head :: tail => 
            val tailSol: Boolean = member(tail, a)
            head == a || tailSol : Boolean
*/

defined function member

In [49]:
def member[A](l: List[A], a: A): Boolean = 
    l.foldRight(false)(
        (head, tailSol) => 
            head == a || tailSol : Boolean)

defined function member

In [50]:
def member[A](l: List[A], a: A): Boolean = 
    l.foldRight(false): (head, tailSol) => 
        head == a || tailSol : Boolean

defined function member

In [45]:
member(List(1,2,3), 5)

res45: Boolean = false

In [42]:
val nil: Int = 0
val cons: (Int, Int) => Int = _ + _ 

(::(1, ::(2, ::(3, Nil)))).foldRight(nil)(cons) == 
cons(1, cons(2, cons(3, nil)))

nil: Int = 0
cons: (Int, Int) => Int = ammonite.$sess.cell42$Helper$$Lambda$3834/219786225@69f30b37
res42_2: Boolean = true

In [15]:
def dyv[A, B](l: List[A])(nil: B, cons: (A, B) => B): B = 
    l match 
        case Nil => nil : B
        case head :: tail => 
            val tailSol: B = dyv(tail)(nil, cons)
            cons(head, tailSol) : B

defined function dyv

In [2]:
// sum of elements
def sum(l: List[Int]): Int = 
    l match 
        case Nil => 0 : Int 
        case head :: tail => 
            val tailSol: Int = sum(tail)
            tailSol + head : Int

defined function sum

In [25]:
// sum of elements
def sum(l: List[Int]): Int = 
    l.foldRight(0)(_+_)

defined function sum

In [24]:
// sum of elements
def sum(l: List[Int]): Int = 
    l.foldRight(0)(_+_)

defined function sum

In [27]:
// sum of elements
def sum(l: List[Int]): Int = 
    l.foldRight[Int](0)((v1: Int, v2: Int) => v1+v2)

defined function sum

In [22]:
// sum of elements
def sum(l: List[Int]): Int = 
    dyv(l)(0, _+_)

defined function sum

In [26]:
// sum of elements
def sum(l: List[Int]): Int = 
    dyv[Int, Int](l: List[Int])(0:Int, 
        (head: Int, tailSol: Int) => head+tailSol)

defined function sum

In [5]:
def last[A](l: List[A]): Option[A] = 
    l match 
        case Nil => None : Option[A]
        case head :: tail => 
            val tailSol: Option[A] = last(tail)
            if tailSol == None then Some(head)
            else tailSol : Option[A]

defined function last

In [20]:
def last[A](l: List[A]): Option[A] = 
    dyv(l)(None : Option[A], 
        (head, tailSol) =>
            if tailSol == None then Some(head)
            else tailSol : Option[A]
    )

defined function last

In [33]:
def last[A](l: List[A]): Option[A] = 
    dyv[A, Option[A]](l)(None : Option[A], 
        (head, tailSol) =>
            if tailSol == None then Some(head)
            else tailSol : Option[A]
    )

defined function last

In [34]:
def last[A](l: List[A]): Option[A] = 
    dyv[A, Option[A]](l)(None, 
        (head, tailSol) =>
            if tailSol == None then Some(head)
            else tailSol : Option[A]
    )

defined function last

In [29]:
def last[A](l: List[A]): Option[A] = 
    l.foldRight(None: Option[A])( 
        (head, tailSol) =>
            if tailSol == None then Some(head)
            else tailSol : Option[A]
    )

defined function last

In [30]:
def last[A](l: List[A]): Option[A] = 
    l.foldRight(None)( 
        (head, tailSol) =>
            if tailSol == None then Some(head)
            else tailSol : Option[A]
    )

-- [E007] Type Mismatch Error: cell31.sc:4:40 ----------------------------------
4 |            if tailSol == None then Some(head)
  |                                    ^^^^^^^^^^
  |      Found:    Some[A]
  |      Required: None.type
  |
  |      The following import might make progress towards fixing the problem:
  |
  |        import sourcecode.Text.generate
  |
  |
  | longer explanation available when compiling with `-explain`
-- [E007] Type Mismatch Error: cell31.sc:5:17 ----------------------------------
5 |            else tailSol : Option[A]
  |                 ^^^^^^^^^^^^^^^^^^^
  |      Found:    Option[A]
  |      Required: None.type
  |
  |      The following import might make progress towards fixing the problem:
  |
  |        import sourcecode.Text.generate
  |
  |
  | longer explanation available when compiling with `-explain`
Compilation Failed

In [32]:
def last[A](l: List[A]): Option[A] = 
    l.foldRight(None: Option[A])( 
        (head, tailSol) =>
            if tailSol == None then Some(head)
            else tailSol : Option[A]
    )

defined function last

In [30]:
def last[A](l: List[A]): Option[A] = 
    l.foldRight(Option.empty[A])( 
        (head, tailSol) =>
            if tailSol == None then Some(head)
            else tailSol : Option[A]
    )

defined function last

In [36]:
def last[A](l: List[A]): Option[A] = 
    l.foldRight(Option.empty[A]):
        case (head, None) => Some(head)
        case (_, tailSol) => tailSol

defined function last

In [38]:
def last[A](l: List[A]): Option[A] = 
    l.foldRight(Option.empty[A])({
        case (head, None) => Some(head)
        case (_, tailSol) => tailSol
    })

defined function last

In [38]:
def last[A](l: List[A]): Option[A] = 
    l.foldRight(Option.empty[A]){
        case (head, None) => Some(head)
        case (_, tailSol) => tailSol
    }

defined function last

In [31]:
def last[A](l: List[A]): Option[A] = 
    l.foldRight[Option[A]](None)( 
        (head, tailSol) =>
            if tailSol == None then Some(head)
            else tailSol : Option[A]
    )

defined function last

In [8]:
def last[A](l: List[A]): Option[A] = 
    l match 
        case Nil => None : Option[A]
        case head :: Nil => Some(head)
        case head :: tail => last(tail)

defined function last

In [9]:
last(List(1,2,3,4,5)) == Some(5)
last(0 :: List(1, 2, 3, 4, 5)) == Some(5)

res9_0: Boolean = true
res9_1: Boolean = true

In [10]:
last(Nil) == None
last(0 :: Nil) == Some(0)

res10_0: Boolean = true
res10_1: Boolean = true

In [3]:
// product of elements 

defined function multiply

These functions clearly share a common logic: their only differences are the value which is returned when the list is empty, and the function used to combine numbers (`+` and `*`, respectively). We can abstract away these differences and arrive to a more generic function which encodes that common logic:

In [4]:
// monomorphic `combine` method

defined function combine

which allows us to re-define in a modular way the `sum` and `multiply` functions:

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [ ]:
def multiply(list: List[Int]): Int = 
    ???

But we don't need to constrain ourselves to integers. In its generic version, the `combine` function is actually the `foldRight` higher-order function (for `List`s):

In [ ]:
// more generically: foldRight


Graphically, we can explain the behaviour of `foldRight` as follows: 

![with elements](../../images/foldRight.1.svg)

(all images credit to [Scala Visual Reference](https://superruzafa.github.io/visual-scala-reference/))

We can understand `foldRight` as an implementation of the divide-and-conquer design pattern: first, divide your problems in subproblems; second, solve these sub-problems; last, compose their solutions to obtain the solution to the overall problem. If the sub-problems are simple enough they can be solved directly; otherwise, they are solved recursively. In the case of lists:
- The problem is obtaining a value of type `B` for a given list.
- The only sub-problem corresponds to the tail of that list. 
- The arguments of the `foldRight` function tell us how to obtain the solution for the empty list (the atomic problem which can not be further decomposed), and how to obtain the solution from the solution to the sub-problem.

From this perspective, the implementation of `sum` and `multiply` is exactly the same as before when we used the function `combine`, although we can explain them differently:

In [6]:
def sum(list: List[Int]): Int = 
    foldRight[Int, Int](list)(
        0, // direct solution to atomic problem
        (head, subsol) => head + subsol // composing sub-solution
    )

defined function sum

In [7]:
def multiply(list: List[Int]): Int = 
    foldRight(list)(
        1, // direct solution
        (head: Int, subsol: Int) => head * subsol // compose
    )

defined function multiply

#### HOFs in the Scala API

The `foldRight` HOF, as all the HOFs that we will explain in this notebook, are actually part of the standard Collections library of Scala. Typically, they are implemented as member methods of the corresponding collection class. For lists, the [Scala API](https://www.scala-lang.org/api/2.13.1/scala/collection/immutable/List.html) offers something like this:

In [9]:
object Standard{
    class List[A]{
        def foldRight[B](b: B)(f: (B, A) => B): B = ???
    }
}

defined object Standard

So, invocations of the standard `foldRight` do not receive the list as argument, but are common method invocations on some `List` instance:

In [10]:
// foldRight over lists



res10: Int = 6

Normally, we will use standard HOFs instead of our home-made versions, once they are explained. 

## The Hall of Fame of HOFs

Besides `compose`, `andThen` and `foldRight`, there are other famous higher-order functions which work great as modularity devices: 

In [11]:
def foldLeft[A, B](list: List[A])(b: B)(f: (B, A) => B): B = ???
def filter[A](list: List[A])(f: A => Boolean): List[A] = ???
def map[A, B](list: List[A])(f: A => B): List[B] = ???
def flatMap[A, B](list: List[A])(f: A => List[B]): List[B] = ???

defined function foldLeft
defined function filter
defined function map
defined function flatMap

But most of these functions do not only work for `List`, they work also for `Option`, `Either[A, ?]`, and many other data structures. For instance:

In [15]:
object Std: 
    enum Option[+A]: 
        case None
        case Some(a: A)

    enum Either[A, B]: 
        case Left(a: A)
        case Right(b: B)

defined object Std

In [16]:
def foldOption[A, B](o: Option[A])(none: B, some: A => B): B = 
    o match 
        case None => none : B
        case Some(a: A) => some(a)

defined function foldOption

In [17]:
def foo(o: Option[String]): Int = 
    o match 
        case None => 0
        case Some(s) => s.length

defined function foo

In [19]:
def bar(o: Option[Int]): Boolean = 
    o match 
        case None => false
        case Some(i) => i > 5 : Boolean

defined function bar

In [20]:
def foo(o: Option[String]): Int = 
    foldOption(o)(
        0,
        (s: String) => s.length
    )

defined function foo

In [21]:
def bar(o: Option[Int]): Boolean = 
    foldOption(o)(
        false,
        (i: Int) => i > 5
    )

defined function bar

In [23]:
def foo(o: Option[String]): Int = 
    // o.fold(0)((s: String) => s.length)
    o.fold(0)(_.length)

defined function foo

In [24]:
def bar(o: Option[Int]): Boolean = 
    o.fold(false)(_ > 5)

defined function bar

In [12]:
def fold[A, B](opt: Option[A])(none: B)(some: A => B): B = ???
def filter[A](opt: Option[A])(f: A => Boolean): Option[A] = ???
def map[A, B](opt: Option[A])(f: A => B): Option[B] = ???  
def flatMap[A, B](opt: Option[A])(f: A => Option[B] ): Option[B] = ???

defined function fold
defined function filter
defined function map
defined function flatMap

In [25]:
def foo(e: Either[String, Int]): Boolean = 
    e match 
        case Left(s) => s != "" : Boolean 
        case Right(i) => i > 0 : Boolean 

defined function foo

In [26]:
def foo(e: Either[String, Int]): Boolean = 
    e.fold(
        s => s != "" : Boolean, 
        i => i > 0 : Boolean 
    )

defined function foo

In [27]:
def foo(e: Either[String, Int]): Boolean = 
    e.fold(_ != "", _ > 0)

defined function foo

In [13]:
def fold[A, B, C](opt: Either[A, B])(left: A => C, right: B => C): C = ???
def filter[A, B](opt: Either[A, B])(f: B => Boolean): Either[A, B] = ???
def map[A, B, C](opt: Either[A, B])(f: B => C): Either[A, C] = ???  
def flatMap[A, B, C](opt: Either[A, B])(f: B => Either[A, C] ): Either[A, C] = ???

defined function fold
defined function filter
defined function map
defined function flatMap

The `fold` function is the catamorphism for `Option` and `Either[A, ?]`, in the same way than `foldRight` is the catamorphism for lists (unfortunately, the concept of catamorphism won't be explained due to lack of time). 

## FoldLeft: a better loop

The `foldLeft` HOF is the functional way to implement common imperative algorithms. For instance:

In [14]:
// length function, imperatively
def length(l: List[A]): Int = 
    def step(out: Int, aux: List[A]): Int = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(???(head, out), tail)

    step(???, l)

defined function length

In [14]:
// length function, imperatively
def length(l: List[A]): Int = 
    def step(out: Int, aux: List[A]): Int = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(out+1, tail)

    step(0, l)

defined function length

In [57]:
// length function, imperatively
def length[A](l: List[A]): Int = 
    l.foldLeft(0)((out, _) => out+1)

defined function length

In [51]:
// reverse function, imperatively
def reverse[A](l: List[A]): List[A] = 
    def step(out: List[A], aux: List[A]): List[A] = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(???(head, out), tail)

    step(???, l)

defined function reverse

In [53]:
// reverse function, imperatively
def iterative[A, B](l: List[A])(initial: B, update: (B, A) => B): B = 
    def step(out: B, aux: List[A]): B = 
        aux match 
            case Nil => out
            case e :: tail => 
                step(update(out, e), tail)

    step(initial, l)

defined function iterative

In [51]:
// reverse function, imperatively
def reverse[A](l: List[A]): List[A] = 
    def step(out: List[A], aux: List[A]): List[A] = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(head :: out, tail)

    step(Nil, l)

defined function reverse

In [54]:
// reverse function, imperatively
def reverse[A](l: List[A]): List[A] = 
    iterative(l)(Nil: List[A], (out, e) => e :: out)

defined function reverse

In [55]:
// reverse function, imperatively
def reverse[A](l: List[A]): List[A] = 
    l.foldLeft(Nil: List[A])((out, e) => e :: out)

defined function reverse

In [56]:
// reverse function, imperatively
def reverse[A](l: List[A]): List[A] = 
    //l.foldLeft(Nil: List[A])(_ prepend _)
    ???

defined function reverse

In [52]:
reverse(List(1,2,3,4))

res52: List[Int] = List(4, 3, 2, 1)

The following implementation abstracts away the differences in the above functions, and declares two additional parameters: the initial value of a mutable variable, and the update function executed in every step of the loop.

In [16]:
// abstraction: foldLeft, imperatively


defined function foldLeft

Similarly, the `foldLeft` function is typically used where a tail-recursive function is needed. Its recursive implementation is naturally tail-recursive:

In [ ]:
@annotation.tailrec
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ??? 

-- [E097] Syntax Error: cell1.sc:2:4 -------------------------------------------
2 |def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
  |    ^
  |TailRec optimisation not applicable, method foldLeft contains no recursive calls
Compilation Failed

Using `foldLeft` we can implement functions at a higher-level of abstraction, i.e. using constructs which are nearer to the problem-domain, without caring about mutable variables, tail-safe recursion, etc. Its use also leads to very concise (and readable!) implementations. For instance, these are (almost) one-liner implementations of some of the functions of the last notebook on recursive functions:

In [18]:
def length[A](list: List[A]): Int = 
    ??? 

defined function length

In [19]:
def sum(list: List[Int]): Int = 
    ???

defined function sum

In [20]:
def reverse[A](list: List[A]): List[A] = 
    ??? 

defined function reverse

But we may also use the `foldRight` function to implement them:

In [21]:
def length[A](list: List[A]): Int = 
    ??? 

defined function length

In [22]:
def sum(list: List[Int]): Int = 
    ???

defined function sum

In [24]:
def reverse[A](list: List[A]): List[A] = 
    ???

defined function reverse

Functionally, both versions are equivalent, but the ones using `foldLeft` are more efficient. In particular, the `foldRight` version of `reverse` has cuadratic complexity. 

#### Implementing `foldRight` with `foldLeft`

The implementation that we gave for `foldRight` was not tail-recursive, so this will blow up the stack:

In [4]:
// blowing up the stack 


1 :: 2 :: 3 :: 4 :: Nil
1 +  2 +  3 +  4 +  0
List(1,2,3,4).foldRight(0)((head, tailSum) => head + tailSum)
List(1,2,3,4).foldRight(0)(_ + _)
List(1,2,3,4).reverse.foldLeft(0)(_ + _)



res4_0: List[Int] = List(1, 2, 3, 4)
res4_1: Int = 10
res4_2: Int = 10
res4_3: Int = 10
res4_4: Int = 10

In order to obtain a better implementation, we may first reverse the list and use `foldLeft` as follows: 

In [5]:
def foldRightTR[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    list.reverse.foldLeft(nil): 
        (out, e) => cons(e, out)

defined function foldRightTR

This implementation works right with big lists:

In [28]:
// check it out


res28: Int = 0

#### Implementing `foldLeft` with `foldRight`

This is also possible, though challenging. We start from the tail-recursive implementation of `foldLeft`. First, we rearrange the arguments so that it now returns a function instead of a plain value of type `B`. 

In [29]:
// foldLeftAux


defined function foldLeftAux

In [9]:
// reverse function, imperatively
@annotation.tailrec
def iterative[A, B](l: List[A])(initial: B, update: (B, A) => B): B = 
    l match 
        case Nil => initial
        case e :: tail => 
            iterative(tail)(update(initial, e), update)

defined function iterative

In [12]:
// reverse function, imperatively
// @annotation.tailrec
def foldLeft[A, B](l: List[A])(update: (B, A) => B): B => B = 
    l.foldRight[B => B](initial => initial)(
        (head, tailSol) => 
            initial => tailSol(update(initial, head)))
        /*
    l match 
        case Nil => initial => initial
        case head :: tail => 
            val tailSol: B => B = iterative(tail)(update)
            initial => tailSol(update(initial, head))*/

defined function foldLeft

In [13]:
// reverse function, imperatively
// @annotation.tailrec
def foldLeft[A, B](l: List[A])(update: (B, A) => B): B => B = 
    l.foldRight((initial: B) => initial)(
        (head, tailSol) => 
            initial => tailSol(update(initial, head)))
        /*
    l match 
        case Nil => initial => initial
        case head :: tail => 
            val tailSol: B => B = iterative(tail)(update)
            initial => tailSol(update(initial, head))*/

defined function foldLeft

In [11]:
// reverse function, imperatively
// @annotation.tailrec
def iterative[A, B](l: List[A])(update: (B, A) => B): B => B = 
    l match 
        case Nil => initial => initial
        case e :: tail => 
            val tailSol: B => B = iterative(tail)(update)
            initial => tailSol(update(initial, e))

defined function iterative

In [8]:
def dyv[A, B](l: List[A])(nil: B, cons: (A, B) => B): B = 
    l match 
        case Nil => nil
        case head :: tail => 
            val tailSol: B = dyv(tail)(nil, cons)
            cons(head, tailSol)

defined function dyv

This was in order to better expose the recursion pattern of `foldRight`. The solution is now obtained simply by noticing that `foldLeft(tail)(update)` is the solution for the tail:

In [30]:
// foldLeftAux with foldRight


defined function foldLeftAux

The original signature can then be implemented as follows: 

In [2]:
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ???

defined function foldLeft

## The `map` HOF

In [31]:
Character.toUpperCase(char2Character('a')) // .getClass // toUpperCase // toString.capitalize.apply(0)

res31: Char = 'A'

In [34]:
capitalize(List('a', 'b', 'c')) == List('A', 'B', 'C')

res34: Boolean = true

In [33]:
def capitalize(l: List[Char]): List[Char] = 
    l match 
        case Nil => Nil : List[Char]
        case head :: tail => 
            val tailSol: List[Char] = capitalize(tail)
            Character.toUpperCase(char2Character(head)) :: tailSol : List[Char]

defined function capitalize

In [37]:
def invert(l: List[Int]): List[Int] = 
    l match 
        case Nil => Nil
        case head :: tail => 
            val tailSol: List[Int] = invert(tail)
            (-1 * head) :: tailSol

defined function invert

In [40]:
def capitalize(l: List[Char]): List[Char] = 
    l.map(c => Character.toUpperCase(char2Character(c)))

defined function capitalize

In [40]:
def capitalize(l: List[Char]): List[Char] = 
    l.map((c: Char) => Character.toUpperCase(char2Character(c)))

defined function capitalize

In [37]:
def invert(l: List[Int]): List[Int] = 
    l match 
        case Nil => Nil
        case head :: tail => 
            val tailSol: List[Int] = invert(tail)
            (-1 * head) :: tailSol

defined function invert

In [44]:
def par(l: List[Int]): List[Boolean] = 
    l.map(_ % 2 == 0)

defined function par

In [43]:
def map[A](l: List[A])(f: A => A): List[A] = 
    l match 
        case Nil => Nil
        case head :: tail => 
            val tailSol: List[A] = map(tail)(f)
            f(head) :: tailSol

defined function map

In [45]:
def map[A, B](l: List[A])(f: A => B): List[B] = 
    l match 
        case Nil => Nil
        case head :: tail => 
            val tailSol: List[B] = map(tail)(f)
            f(head) :: tailSol

defined function map

In [46]:
def map[A, B](l: List[A])(f: A => B): List[B] = 
    l.foldRight(Nil: List[B]): 
        (head, tailMapped) => 
            f(head) :: tailMapped

defined function map

In [42]:
def invert(l: List[Int]): List[Int] = 
    l.map(-1 * _)

defined function invert

In [41]:
def invert(l: List[Int]): List[Int] = 
    l.map(head => -1 * head)

defined function invert

In [39]:
def invert(l: List[Int]): List[Int] = 
    l.foldRight(Nil: List[Int]): 
        (head, tailSol) => (-1 * head) :: tailSol

defined function invert

In [38]:
invert(List(-1, 3, -4, -5, 2))

res38: List[Int] = List(1, -3, 4, 5, -2)

In [36]:
def capitalize(l: List[Char]): List[Char] = 
    l.foldRight(Nil : List[Char]): 
        (head, tailSol) => 
            Character.toUpperCase(char2Character(head)) :: tailSol

defined function capitalize

The `map` function is one of the essential HOFs. Its purpose is applying a function to the elements of a data structure, in such a way that the relationships between these elements do not change. The only thing that must be modified is the _content_ of the data structure, not its _shape_. This condition is expressed in the following equations:

1. `map(list)(identity) == list` for all `list: List[A]`
2. `map(map(list)(f))(g) == map(list)(g compose f)` for all `list: List[A]`, `f: A => B`, `g: B => C`

![map](../../images/map.svg)

The implementation for lists can be done as follows:

In [ ]:
// map example 1


In [2]:
// map example 2


In [2]:
// map generically


The `map` function is polymorphic in `A` and `B`, but we can't pass generic functions (aka polymorphic values) as parameters using `FunctionN`classes (these classes only wrap functions from specific types to specific result types). We may create polymorphic versions of `FunctionN` clases, but in order to keep things simple, we will define the test catalogue for `map` using a specific signature chosen at random (any other may serve as well):

In [32]:
class TestMap(
    map: List[Int] => (Int => Boolean) => List[Boolean]
) extends AnyFlatSpec with should.Matchers:

    val isEven: Int => Boolean = _ % 2 == 0

    "mapping an empty list" should "return an empty list" in:
        map(List())(isEven) shouldBe ???

    "mapping an non-empty list" should "only change its content" in:
        map(List(1))(isEven) shouldBe ???
        map(List(1,2))(isEven) shouldBe ???
        map(List(1,2,3))(isEven) shouldBe ???

defined class TestMap

To run the tests we will pass the `map` function for `Ìnt` and `Boolean` types:

In [34]:
run(TestMap(map[Int, Boolean]))

cell32$Helper$TestMap:
mapping an empty list
- should return an empty list
mapping an non-empty list
- should only change its content


although it's not necessary to write the types explicitly (Scala will infer the types for us):

In [35]:
run(TestMap(map))

cell32$Helper$TestMap:
mapping an empty list
- should return an empty list
mapping an non-empty list
- should only change its content


The implementation of `map` is really close to the implementation of `foldRight`. Indeed, we can give a more modular implementation using this more basic HOF: 

In [36]:
/*
def foldRight[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, foldRight(tail)(nil)(cons))
    }
*/

def mapFR[A, B](list: List[A])(f: A => B): List[B] = 
    ???

defined function mapFR

In [37]:
run(TestMap(mapFR))

cell32$Helper$TestMap:
mapping an empty list
- should return an empty list
mapping an non-empty list
- should only change its content


### `map` for `Option`

The implementation of the `map` function given above works over lists, but we can also map optional values and many other data structures. The implementation for the data structure `Option` must satisfy the following test:

In [38]:
class TestMapOption(
    map: Option[Int] => (Int => Boolean) => Option[Boolean]
) extends AnyFlatSpec with should.Matchers:
    
    val isEven: Int => Boolean = _ % 2 == 0

    "mapping the `None` value" should "return `None`" in:
        map(None)(isEven) shouldBe ???
    
    "mapping a `Some` value" should "only change its content" in:
        map(Option(3))(isEven) shouldBe ???
        map(Option(1))(isEven) shouldBe ???
        map(Option(2))(isEven) shouldBe ???

defined class TestMapOption

The implementation is simple:

In [39]:
def mapOpt[A, B](maybeA: Option[A])(f: A => B): Option[B] = 
    ???

defined function mapOpt

In [40]:
run(TestMapOption(mapOpt))

cell38$Helper$TestMapOption:
mapping the `None` value
- should return `None`
mapping a `Some` value
- should only change its content


## Filtering elements

Unlike `map`, the `filter` HOF allows us to change the shape of the data structure, removing those elements that do not satisfy a given predicate. 

![filter](../../images/filter.svg)

In [49]:
ints(List(3.4, 2.0, 8.7, 2.2, 1.0)) == List(2.0, 1.0)

res49: Boolean = true

In [55]:
positives(List(-3, -1, 3, -2, 0, 1))

res55: List[Int] = List(3, 1)

In [56]:
def positives(l: List[Int]): List[Int] = 
    l.filter(head => head > 0)

defined function positives

In [57]:
def ints(l: List[Double]): List[Double] = 
    l.filter(head => head.isValidInt)

defined function ints

In [58]:
def filter[A](l: List[A])(pred: A => Boolean): List[A] = 
    l match 
        case Nil => Nil
        case head :: tail => 
            val tailSol: List[A] = filter(tail)(pred)
            if pred(head) then head :: tailSol
            else tailSol

defined function filter

In [53]:
def positives(l: List[Int]): List[Int] = 
    l match 
        case Nil => Nil
        case head :: tail => 
            val tailSol: List[Int] = positives(tail)
            if head > 0 then head :: tailSol
            else tailSol

defined function positives

In [51]:
def ints(l: List[Double]): List[Double] = 
    l match 
        case Nil => Nil
        case head :: tail if head.isValidInt => 
            head :: ints(tail)
        case _ :: tail => ints(tail)

defined function ints

In [51]:
def ints(l: List[Double]): List[Double] = 
    l match 
        case Nil => Nil
        case head :: tail if head.isValidInt => 
            head :: ints(tail)
        case _ :: tail => ints(tail)

defined function ints

In [50]:
def ints(l: List[Double]): List[Double] = 
    l.foldRight(Nil: List[Double]): 
        (head, tailSol) => 
            if head.isValidInt then head :: tailSol
            else tailSol

defined function ints

In [52]:
def ints(l: List[Double]): List[Double] = 
    l.foldRight(Nil: List[Double]): 
        case (head, tailSol) if head.isValidInt => tailSol
        case (_, tailSol) => tailSol

defined function ints

In [41]:
// filter, generically 


defined function filterR

In [42]:
class TestFilterList(
    filter: List[Int] => (Int => Boolean) => List[Int]
) extends AnyFlatSpec with should.Matchers:
    
    val isEven: Int => Boolean = _ % 2 == 0

    "filter" should "work" in:
        filter(List())(isEven) shouldBe ???
        filter(List(1))(isEven) shouldBe ???
        filter(List(1,3,5))(isEven) shouldBe ???
        filter(List(2,4,6))(isEven) shouldBe ???

defined class TestFilterList

In [43]:
run(TestFilterList(filterR))

cell42$Helper$TestFilterList:
filter
- should work


Using `foldRight` we can get a more modular implementation: 

In [45]:
def filterFR[A](list: List[A])(predicate: A => Boolean): List[A] = 
    ???

defined function filterFR

In [46]:
run(TestFilterList(filterFR))

cell42$Helper$TestFilterList:
filter
- should work


### Filtering optional values

We only have a possible value, so implementing filtering is easy in this case:

In [48]:
class TestFilterOption(
    filter: Option[Int] => (Int => Boolean) => Option[Int]
) extends AnyFlatSpec with should.Matchers:
    
    val isEven: Int => Boolean = _ % 2 == 0

    "filter" should "work" in:
        filter(None)(isEven) shouldBe ???
        filter(Some(1))(isEven) shouldBe ???
        filter(Some(2))(isEven) shouldBe ???

defined class TestFilterOption

In [49]:
def filter[A](maybeA: Option[A])(predicate: A => Boolean): Option[A] = 
    ???

defined function filter

In [50]:
run(TestFilterOption(filter))

cell48$Helper$TestFilterOption:
filter
- should work


## FlatMapping data structures

Let's consider a paragraph represented as a list of sentences, where each sentence is in turn modelled as a string made of _words_ separated by blank spaces. We want to obtain a list of all the words in each sentence. 

In [52]:
class TestWords(
    words: List[String] => List[String]
) extends AnyFlatSpec with should.Matchers:
            
    val paragraph1 = List(
        "En un lugar",
        "de la Mancha", 
        "de cuyo nombre no",
        "quiero acordarme")
    
    "words" should "work" in:
        words(paragraph1) shouldBe 
            ???

defined class TestWords

We may try to map the paragraph with a function that `split`s  each sentence into the words they are made of:

In [53]:
val paragraph: List[String] = List("en un lugar", "de la Mancha")

???

paragraph: List[String] = List("en un lugar", "de la Mancha")
res53_1: List[List[String]] = List(
  List("en", "un", "lugar"),
  List("de", "la", "Mancha")
)

but then we obtain a _list of lists_ of strings, not a list of plain strings. The solution is not far though: we have just to concatenate all the lists and we obtain what we need. The function `flatten` performs this concatenation:

In [54]:
// flatten


defined function flatten

The function `flatten` is actually provided by the Scala API. So, the implementation of the `words` function is as follows:

In [55]:
def words(paragraph: List[String]): List[String] = 
    ???

defined function words

In [57]:
run(TestWords(words))

cell52$Helper$TestWords:
words
- should work


This combination of the HOF `map` and the function `flatten` is so common, that it has been given a proper name: `flatMap`. 

In [58]:
// flatMap 


defined function flatMap

![filter](../images/flatMap.svg)

Using `flatMap` the word function is implemented even more easily:

In [59]:
def words(paragraph: List[String]): List[String] = 
    ???

defined function words

### FlatMapping optional values

In [60]:
def flatMap[A, B](maybeA: Option[A])(cont: A => Option[B]): Option[B] = 
    ???

defined function flatMap

In [61]:
class TestFlatMapOption(
    flatMap: Option[Int] => (Int => Option[Boolean]) => Option[Boolean]
) extends AnyFlatSpec with should.Matchers:
    
    def isPositiveEven(i: Int): Option[Boolean] = 
        if (i>=0) Some(i % 2 == 0)
        else None
    
    "flatMap" should "work" in:
        flatMap(None)(isPositiveEven) shouldBe ???
        flatMap(Some(5))(isPositiveEven) shouldBe ???
        flatMap(Some(-5))(isPositiveEven) shouldBe ???
        flatMap(Some(0))(isPositiveEven) shouldBe ???
        flatMap(Some(4))(isPositiveEven) shouldBe ???

defined class TestFlatMapOption

In [62]:
run(TestFlatMapOption(flatMap))

cell61$Helper$TestFlatMapOption:
flatMap
- should work


## All together: a modularity problem

As before, we start from a list of sentences, but now we want to take care of the possible extra spaces between each word. Also, we want to obtain not the words themselves but their lengths. So, the specification of our problem is a function `lengths`:

In [64]:
class TestLengths(
    lengths: List[String] => List[Int]
) extends AnyFlatSpec with should.Matchers:
            
    val paragraph1 = List(
        "En un  lugar",
        "de  la Mancha ", 
        "de cuyo nombre no",
        "quiero        acordarme")
    
    "lengths" should "work" in:
        lengths(paragraph1) shouldBe 
            List(2, 2, 5, 2, 2, 6, 
                 2, 4, 6, 2, 6, 9)

defined class TestLengths

We may try to implement this function imperatively using mutable variables:

In [70]:
def lengthsM(paragraph: List[String]): List[Int] =
    ???

defined function lengthsM

and this works, of course: 

In [69]:
run(TestLengths(lengthsM))

cell64$Helper$TestLengths:
lengths
- should work


but can we do it better? Yes, we can! Using HOFs we can get a more concise, understandable and reliable version, with a great level of reuse!

In [71]:
def lengthsHOF(paragraph: List[String]): List[Int] = 
    ???

defined function lengthsHOF

In [73]:
run(TestLengths(lengthsHOF))

cell64$Helper$TestLengths:
lengths
- should work


This solution contrasts with the mutable version in several respects:
- It's more **modular**, i.e. it's made of coarse-grained components: the HOFs `flatMap`, `filter` and `map`. The mutable version builds instead upon fine-grained components: `var`s, and `for` loops.
- The HOF components `flatMap`, etc., are actually very generic and domain-independent, and are typically part of standard libraries, so the level of **reuse** and **reliability** of the HOF-based solution is very high.
- The HOF-based solution is more **understandable**: it models the solution to the problem in terms of standard _transformations_ which are composed together using plain function composition. Moreover, these transformations are also at the right level of abstraction, i.e. it's natural to specify the solution to the problem in terms of flatMap, filter and map.

### For-comprehensions

The combination of `map`, `flatMap` and `filter` HOFs is so common, that Scala has a special syntax for them: for-comprehensions.

In [3]:
def lengthsFC(paragraph: List[String]): List[Int] = 
    ???

defined function lengthsFC

In [77]:
run(TestLengths(lengthsFC))

cell64$Helper$TestLengths:
lengths
- should work
